In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
xyz = [[-875, 0, 9.755], [442, 0, 9.755], [1921, 0, 9.755], [2951, 0.5, 9.755], [-4132, 0.5, 23.618],
[-876, 0, 23.618]]
uv = [[76, 706], [702, 706], [1440, 706], [1867, 706], [264, 523], [625, 523]]

In [3]:
control_pt=np.array(xyz)
image_pt=np.array(uv)

In [4]:
def normalize(point):
    mean=np.mean(point,axis=0)
    s=2**0.5*point.shape[0]/np.sqrt(np.sum(np.square(point-mean)))
    
    if point.shape[1]==3:
        mat=[[s,0,0,-s*mean[0]],[0,s,0,-s*mean[1]],[0,0,s,-s*mean[2]],[0,0,0,1]]
    elif point.shape[1]==2:
        mat=[[s,0,-s*mean[0]],[0,s,-s*mean[1]],[0,0,1]]
    
    mat=np.array(mat)
    mat=np.linalg.inv(mat)
    comp_arr=np.concatenate((point,np.ones(shape=(point.shape[0],1))),axis=1)
    norm=np.dot(comp_arr,mat)
    return mat,norm

In [5]:
mat_3D,control_pt_norm=normalize(control_pt)
#control_pt_norm=control_pt_norm/control_pt_norm[:,-1].reshape(-1,1)
control_pt_norm= control_pt_norm[:,:-1]

mat_2D,image_pt_norm=normalize(image_pt)
#image_pt_norm=image_pt_norm/image_pt_norm[:,-1].reshape(-1,1)
image_pt_norm= image_pt_norm[:,:-1]

In [6]:
def calculate_dlt(pt_3,pt_2):
    A=[]
    for i in range(pt_3.shape[0]):
        arr1=[-pt_3[i][0],-pt_3[i][1],-pt_3[i][2],-1,0,0,0,0,pt_3[i][0]*pt_2[i][0],pt_3[i][1]*pt_2[i][0],pt_3[i][2]*pt_2[i][0],pt_2[i][0]]
        A.append(arr1)
        arr1=[0,0,0,0,-pt_3[i][0],-pt_3[i][1],-pt_3[i][2],-1,pt_3[i][0]*pt_2[i][1],pt_3[i][1]*pt_2[i][1],pt_3[i][2]*pt_2[i][1],pt_2[i][1]]
        A.append(arr1)
    
    A=np.array(A)
    
    U,sigma,V=np.linalg.svd(A)
    P=V[-1,:]
    H = P.reshape(3, pt_3.shape[1] + 1)
    H=np.dot(np.dot(np.linalg.inv(mat_2D),H),mat_3D)
    return H,sigma

In [7]:
Homo,sigma=calculate_dlt(control_pt_norm,image_pt_norm)

In [8]:
pt=np.concatenate((control_pt,np.ones(shape=(control_pt.shape[0],1))),axis=1)

In [9]:
np.sum(Homo[1]*pt[-1])/np.sum(Homo[-1]*pt[-1])

519.434195005755

In [20]:
H_inf=Homo[:,:3].reshape(-1,3)
h_vec=Homo[:,-1].reshape(-1,1)

In [24]:
X_o=np.dot(np.linalg.inv(H_inf),h_vec)

In [27]:
R_inv,K_inv=np.linalg.qr(np.linalg.inv(H_inf))

In [28]:
R=R_inv.T
K=np.linalg.inv(K_inv)
K=K/K[-1,-1]

In [29]:
K

array([[-1.80240606e+00,  1.03705132e+03,  1.19028284e+03],
       [ 0.00000000e+00,  8.10986810e+01,  5.03812935e+02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [30]:
R

array([[-9.99999988e-01,  1.53562549e-04, -3.96416711e-11],
       [-1.42174604e-04, -9.25841735e-01, -3.77911447e-01],
       [-5.80330818e-05, -3.77911442e-01,  9.25841746e-01]])